## Copies data from blob to local host

##### Copyright (C) Microsoft Corporation.  
see license file for details 

In [1]:
# Allow multiple displays per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# AZUREML_NATIVE_SHARE_DIRECTORY mapping to host dir is set by _nativeSharedDirectory_ in .compute file 

import os
try:
    amlWBSharedDir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']    
except:
    amlWBSharedDir =  os.getcwd()
    print('not using AML Workbench?')
    
amlWBSharedDir

not using AML Workbench?


'/local_dir/code'

In [4]:
# Use the Azure Machine Learning data collector to log various metrics

# from azureml.logging import get_azureml_logger
# logger = get_azureml_logger()

In [5]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [6]:
# import utlity functions

import sys, os
paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['src'])))]
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

[None]

In [7]:
# create the file path variables 
# create nih_chest_xray_data_dir (in container dir mapped to a host dir for data persistence), 
# where data will be copied from blob

prj_consts = azure_chestxray_utils.chestxray_consts()

os.getcwd()


data_base_input_dir=os.path.join(amlWBSharedDir, os.path.join(*(['..']+prj_consts.BASE_INPUT_DIR_list)))
credential_info_path=os.path.join(amlWBSharedDir, os.path.join(*(prj_consts.CREDENTIALS_DIR_list)))
nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))

!mkdir -p {data_base_input_dir}
!mkdir -p {credential_info_path}  
!mkdir -p {nih_chest_xray_data_dir}

data_base_input_dir
credential_info_path
nih_chest_xray_data_dir

'/local_dir/code'

'/local_dir/code/../chestxray/data/ChestX-ray8'

'/local_dir/code/notShared'

'/local_dir/code/../chestxray/data/ChestX-ray8/ChestXray-NIHCC'

In [8]:
# Create this function in file '{credential_info_path}/get_data_access_secrets.py'
# def get_blob_credentials():
#     dataBlob = 'https://somedatastore.blob.core.windows.net/somecontainer'
#     sourceKey = 'somesourceKey'
#     return dataBlob, sourceKey
# 
# For example:
if 0:
    CredentialsFileName = os.path.join(credential_info_path, 'get_data_access_secrets.py') 
    crt_container = 'https://somedatastore.blob.core.windows.net/somecontainer'
    crt_container = 'https://ghiordanchestxrbait02sa.blob.core.windows.net:443/chestxraydata'
    crt_source_Key = 'QAMVc1skSmsovoYyycyeP4Oq4pzhwHEIVMqFsuIMg6TbF0LcnRYBsPCPPyHGmhaRrhlruhACO+DWJuzZTOmctg=='

    import os.path 

    if not os.path.isfile(CredentialsFileName): 
        print('Credentials file not found, will be written!') 
        with open(CredentialsFileName,'w') as myFile:
            myFile.write(("def get_blob_credentials():\n\t" +
            "dataBlob = '" + crt_container + "'\n\t" +
            "sourceKey = '" + crt_source_Key + "'\n\t" +
            "return dataBlob, sourceKey\n" )) 
        print(('Credentials file written at ' +CredentialsFileName)) 
    else: 
        print(('Credentials file found at ' + CredentialsFileName))
    
# check CredentialsFileName existence and print content if needed

credentials_file_name = os.path.join(*([credential_info_path, 'get_data_access_secrets.py']))
credentials_file_name
# !cat {credentials_file_name}

'/local_dir/code/notShared/get_data_access_secrets.py'

In [9]:
# read credentials info

add_path_to_sys_path(credential_info_path)         
import get_data_access_secrets   
crt_container, crt_key = get_data_access_secrets.get_blob_credentials()
crt_container
crt_key

'https://ghiordanchestxrbait02sa.blob.core.windows.net:443/chestxraydata'

'QAMVc1skSmsovoYyycyeP4Oq4pzhwHEIVMqFsuIMg6TbF0LcnRYBsPCPPyHGmhaRrhlruhACO+DWJuzZTOmctg=='

In [10]:
#check azcopy is available. Uncomment second line to print _response_ if needed

response = !azcopy
response[1:10]

['azcopy 7.1.0-netcorepreview Copyright (c) 2017 Microsoft Corp. All Rights Reserved.',
 '------------------------------------------------------------------------------',
 '# azcopy is designed for high-performance uploading, downloading, and copying',
 'data to and from Microsoft Azure Blob, and File storage.',
 '',
 '# Command Line Usage:',
 '    azcopy --source <source> --destination <destination> [options]',
 '',
 '# Options:']

In [11]:
tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')
tmp_dir
!mkdir - p $tmp_dir

'/local_dir/code/../chestxray/data/ChestX-ray8/ChestXray-NIHCC/tmp'

In [12]:
# copy data to local host dir
# add {  --source-key {crt_key}  } if needed

tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')

answer = !yes | azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir} \
    --source-key {crt_key} \
    --recursive \
    --verbose \
    --include "images_"


 The above cell will take about 20 minutes to run. While it runs, you can monitor the progress by ssh-ing 
 into your compute context machine and running:   
 find /datadrive01/amlwbShare/current_exp_account/current_workspace/current_exp/chestxray/data/ChestX-ray8/ChestXray-NIHCC/ -type f | wc -l

In [13]:
answer[-7:]

['[2018/09/24 02:52:14] Transfer summary:',
 '-----------------',
 'Total files transferred: 12',
 'Transfer successfully:   4',
 'Transfer skipped:        0',
 'Transfer failed:         8',
 'Elapsed time:            00.00:16:07']

In [14]:
nih_chest_xray_data_dir
!ls -l $tmp_dir
# !find $nih_chest_xray_data_dir -type f | wc -l

'/local_dir/code/../chestxray/data/ChestX-ray8/ChestXray-NIHCC'

total 28835536
-rw-r--r-- 1 keras users  226840150 Sep 24 02:36 images_001.tar.gz
-rw-r--r-- 1 keras users  226610774 Sep 24 02:36 images_002.tar.gz
-rw-r--r-- 1 keras users  289443414 Sep 24 02:36 images_003.tar.gz
-rw-r--r-- 1 keras users 3838903983 Sep 24 02:37 images_004.tar.gz
-rw-r--r-- 1 keras users 3935496531 Sep 24 02:37 images_005.tar.gz
-rw-r--r-- 1 keras users 3986301172 Sep 24 02:37 images_006.tar.gz
-rw-r--r-- 1 keras users 4016328426 Sep 24 02:37 images_007.tar.gz
-rw-r--r-- 1 keras users 4018347353 Sep 24 02:37 images_008.tar.gz
-rw-r--r-- 1 keras users 4111327929 Sep 24 02:37 images_009.tar.gz
-rw-r--r-- 1 keras users 4181556296 Sep 24 02:37 images_010.tar.gz
-rw-r--r-- 1 keras users 4187084020 Sep 24 02:37 images_011.tar.gz
-rw-r--r-- 1 keras users 2914187733 Sep 24 02:37 images_012.tar.gz


In [15]:
!echo ls -l $nih_chest_xray_data_dir
!cd $nih_chest_xray_data_dir
# ! ls -l $nih_chest_xray_data_dir/*.tar.gz 
# ! cat $nih_chest_xray_data_dir/*.tar.gz | tar -zxvf - -i
!for file in $tmp_dir/*.tar.gz; do tar -zxf $file; done


ls -l /local_dir/code/../chestxray/data/ChestX-ray8/ChestXray-NIHCC
tar (child): /*.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [17]:
!jupyter nbconvert --to html 001_get_data.ipynb

[NbConvertApp] Converting notebook 001_get_data.ipynb to html
[NbConvertApp] Writing 275149 bytes to 001_get_data.html
